In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

In [2]:
new_shoreline = gpd.GeoDataFrame.from_features({"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[176.937701,-39.336281],[176.936542,-39.336397],[176.935598,-39.33731]]}}]}, crs="EPSG:4326")
new_shoreline

,geometry
0,"LINESTRING (176.9377 -39.33628, 176.93654 -39...."


In [3]:
new_shoreline.explore(tiles="ESRI.WorldImagery")

In [4]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [5]:
nz_shorelines = shorelines[shorelines.id.str.startswith("nzd")]
latest_siteid = nz_shorelines.id.max()
new_siteid = f"nzd0{int(latest_siteid[4:])+1}"
print(f"Latest siteid is {latest_siteid}, so this new site will be {new_siteid}")
new_shoreline["id"] = new_siteid
new_shoreline

Latest siteid is nzd0562, so this new site will be nzd0563


,geometry,id
0,"LINESTRING (176.9377 -39.33628, 176.93654 -39....",nzd0563


In [6]:
pd.concat((shorelines, new_shoreline)).to_file("shorelines.geojson", driver="GeoJSON")

In [7]:
new_poly = gpd.GeoDataFrame([{"id": new_siteid}], geometry=new_shoreline.to_crs(2193).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_poly.explore()

In [8]:
pd.concat((poly, new_poly)).to_file("polygons.geojson", driver="GeoJSON")

In [9]:
nz_transects = transects[transects.id.str.startswith("nzd")].to_crs(2193)
nz_transects

,id,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,geometry
95050,nzd0001-0000,nzd0001,359.037136,3197.737936,1.000000,0.050,0.0397,0.0679,-0.136498,184.0,143.0,0.001809,19.590361,572.322498,23.923263,325.377492,"LINESTRING (1596662.375 6190263.089, 1596646.6..."
95051,nzd0001-0001,nzd0001,359.037136,3097.737936,0.968728,NaN,NaN,NaN,-0.316164,184.0,157.0,0.011511,17.845204,490.058427,22.137263,330.472572,"LINESTRING (1596744.903 6190264.499, 1596729.1..."
95052,nzd0001-0002,nzd0001,359.037136,2997.737936,0.937456,0.060,0.0478,0.0776,-0.412752,184.0,160.0,0.024944,16.023083,374.900676,19.362352,336.564138,"LINESTRING (1596827.431 6190265.908, 1596811.6..."
95053,nzd0001-0003,nzd0001,359.037136,2897.737936,0.906184,NaN,NaN,NaN,-0.469545,184.0,161.0,0.040228,14.359386,299.451117,17.304656,345.186311,"LINESTRING (1596909.959 6190267.317, 1596894.2..."
95054,nzd0001-0004,nzd0001,359.037136,2797.737936,0.874912,0.055,0.0450,0.0664,-0.531422,184.0,162.0,0.055976,13.451432,276.975375,16.642577,357.606234,"LINESTRING (1596992.486 6190268.724, 1596976.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127421,nzd0562-0014,nzd0562,NaN,NaN,NaN,0.100,NaN,NaN,0.262019,583.0,447.0,0.043431,6.676504,80.625540,8.979173,193.845690,"LINESTRING (1770293.383 5917676.929, 1770648.7..."
127422,nzd0562-0015,nzd0562,NaN,NaN,NaN,0.100,NaN,NaN,0.158424,583.0,448.0,0.015499,6.952012,84.615902,9.198690,196.070192,"LINESTRING (1770254.942 5917751.29, 1770610.27..."
127423,nzd0562-0016,nzd0562,NaN,NaN,NaN,0.100,NaN,NaN,0.130158,583.0,439.0,0.011180,6.628118,79.216921,8.900389,205.098465,"LINESTRING (1770214.898 5917828.821, 1770573.4..."
127424,nzd0562-0017,nzd0562,NaN,NaN,NaN,0.100,NaN,NaN,0.005730,583.0,405.0,0.000014,8.118279,126.880731,11.264135,209.805326,"LINESTRING (1770180.582 5917901.338, 1770542.2..."


In [10]:
def create_transects(line, spacing=80, transect_length=400):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ])
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=2193).to_crs(4326)
    return transects

new_transects = create_transects(new_shoreline.to_crs(2193).geometry.iloc[0])
display(new_transects)
m = new_transects.explore()
new_shoreline.explore(m=m)
new_poly.boundary.explore(m=m)
gpd.GeoSeries(new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry
0,"LINESTRING (176.9374 -39.3345, 176.938 -39.33807)"
1,"LINESTRING (176.93648 -39.33459, 176.93708 -39..."
2,"LINESTRING (176.93431 -39.33569, 176.93792 -39..."


Make sure the origin is inland


In [ ]:
new_transects["id"] = new_siteid + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
new_transects["site_id"] = new_siteid
#new_transects["beach_slope"] = .1
new_transects

,geometry,id,site_id,beach_slope
0,"LINESTRING (176.9374 -39.3345, 176.938 -39.33807)",nzd0563-0000,nzd0563,0.1
1,"LINESTRING (176.93648 -39.33459, 176.93708 -39...",nzd0563-0001,nzd0563,0.1
2,"LINESTRING (176.93431 -39.33569, 176.93792 -39...",nzd0563-0002,nzd0563,0.1


In [12]:
pd.concat((transects, new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")